In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from googletrans import Translator
from analysis_helpers import get_correlations
import simplemma

In [2]:
# Read the preprocessed dataset
df = pd.read_parquet('../data/used_car_dataset.parquet')
df.head()

,price (HUF),condition,design,vehicle history,financing,clock position (km),shippable persons number,number of doors,color,own weight (kg),...,hard drive,hdmi input,door actuator,reversing camera input,chrome rims,1 din,sliding door,leather upholstery,double-sided sliding door,gesture control
0,6895000,Kitűnő,Sedan,Elérhető autó-előélet alvázszám alapján,25%-tól elvihető,110000,5.0,4.0,Fekete (metál),1426.0,...,False,False,False,False,False,False,False,False,False,False
1,9999999,Megkímélt,Városi terepjáró (crossover),Elérhető autó-előélet alvázszám alapján,20%-tól elvihető,119300,5.0,5.0,Fehér,2150.0,...,False,False,False,False,False,False,False,False,False,False
2,4995000,Kitűnő,Városi terepjáró (crossover),Elérhető autó-előélet alvázszám alapján,20%-tól elvihető,70000,5.0,5.0,Kék,1200.0,...,False,False,False,False,False,False,False,False,False,False
3,3499000,Kitűnő,Városi terepjáró (crossover),Elérhető autó-előélet alvázszám alapján,,100000,5.0,5.0,Sötétkék,1176.0,...,False,False,False,False,False,False,False,False,False,False
4,12990000,Normál,Coupe,Elérhető autó-előélet alvázszám alapján,20%-tól elvihető,47336,5.0,4.0,,1490.0,...,False,False,False,False,False,False,False,True,False,False


### Get car brand

In [3]:
# Add brand from link
df['brand'] = [x[4] for x in df.link.str.split('/')]
df['brand'].head()

0       volkswagen
1    mercedes-benz
2            dacia
3          renault
4    mercedes-benz
Name: brand, dtype: object

In [4]:
# Color contains 2 peaces of inprmation the exact color and the is the car has metallic polish or not
df['metallic_polish'] = df['color'].str.contains('metál')
df['color'] = df['color'].str.replace(' (metál)', '', regex=False).str.lower()

### Extract the city from the address

If the car is advertised by a shop the first contact information is the name of the shop. If the car advertised by an individual then the first one is the address. If wh have the address then we can extract the city name from it. Luckily we have a csv with all the hungarian city names so we can find them in the addresses even if the format of the address is not uniform so we would struggle with writing a regex from this

In [5]:
df_hun_sattlements = pd.read_csv('all_hun_settlement.csv')

df_hun_sattlements['cleaned_settlement'] = df_hun_sattlements['settlement'].str.replace(r'Budapest.+', 'Budapest').str.lower().str.strip()
#df_hun_sattlements.drop_duplicates('cleand_settlement', inplace=True)

df_hun_sattlements.head()

/var/folders/6p/rn1y8jlj4q5fp4y6jrn3qslc0000gn/T/ipykernel_53055/2543547996.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_hun_sattlements['cleaned_settlement'] = df_hun_sattlements['settlement'].str.replace(r'Budapest.+', 'Budapest').str.lower().str.strip()


,zip,settlement,short,latitude,longitude,cleaned_settlement
0,1011,Budapest I. kerület,BU,47.5011,19.0469,budapest
1,1012,Budapest I. kerület,BU,47.5000,19.0833,budapest
2,1013,Budapest I. kerület,BU,47.4961,19.0309,budapest
3,1014,Budapest I. kerület,BU,47.5034,19.0349,budapest
4,1015,Budapest I. kerület,BU,47.5000,19.0833,budapest


In [6]:
def get_city_from_zip(_zip):
    city = df_hun_sattlements[df_hun_sattlements.zip == _zip]['cleaned_settlement']
    if len(city) > 0:
        return city.values[0]

In [7]:
msk_is_salse_shop = df['buy_from_shop']
# If the car is advertised by a shop the first contact information is the name of the shop
# If the car advertised by an individual then the first one is the address
# First get the addresses in 1 column
address = pd.Series(np.where(msk_is_salse_shop, df.content_info_1, df.content_info_0))

# Convert it to lower case since we converted all the city names as well
address = address.str.lower()

# Remove "looking in map" additinal text that come form the scraping not part of the address
address = address.str.replace('térkép megtekintése', '', regex=False)

zip_code = address.str.replace('[^\d ]', '', regex=True)
zip_code = zip_code.str.extract("( \d\d\d\d )")[0].astype(float)
cities_based_on_zip = [
    get_city_from_zip(x) for x in zip_code
]

address_without_numbers = address.str.replace("[^a-záéíóöőúüű\s]", "", regex=True)

# Extract the city name from each address
df['city'] = address_without_numbers.str.extract(f"( {' | '.join(df_hun_sattlements['cleaned_settlement'].unique())} )")
df['city'] = df['city'].str.strip()

df['city'] = np.where(df['city'].isna(), cities_based_on_zip, df['city'])

In [8]:
# A lot of advertisemnt don't show address
df['city'].isna().mean()

0.3153458415009801

### Get take away discount values

In [9]:
# Take aways from X%
# Drop taken away from variable
df['financing'] = df['financing'].str.replace(r'\D', '', regex=True).replace({'':'100'}).astype(int)

In [10]:
df['financing'].value_counts()

100    53955
20      9913
30      3155
25      2674
0        843
40       347
35       282
50       163
10        45
45        27
15        12
60         3
70         1
Name: financing, dtype: int64

In [11]:
cols = df.columns
# Drop taken aw columsn to remove duplicated information
cols[cols.str.lower().str.contains('take')]

Index(['Take away from 20%', 'Can be taken away from 30%',
       'Take away from 25%'],
      dtype='object')

In [12]:
# Use years instead of days becuase this high resulution of the age doeasn't really matter on the other hand makes if harder to biuld model
df['age_year'] = (df['age_days']/365).astype(int)

### Get freatures from description# Remuve unneccessary spaces

(1. **Sentiment Analysis:**  **It would work only with english text! Do translation before!** You can perform sentiment analysis on the description to see if it's generally positive, negative, or neutral. This could be interesting to see if there's any correlation between the sentiment of the description and other variables.)

2. **Text Length:** The length of the description could be indicative of something. For instance, sellers who are more serious might write more detailed descriptions. You can easily calculate this with Python's `len()` function.

3. **Word Count:** Similar to text length, but this might give different results because it doesn't count characters but words. A higher word count might indicate a more complex description.

4. **Count of Specific Words:** You can count the occurrence of specific words that you think might be important. For example, if you're looking at car listings, words like "new", "used", "clean", "serviced" etc., might be of interest.

5. **Count of Capitalized Words:** Sellers might capitalize certain words to draw attention to them. Counting the number of capitalized words could be another feature.

6. **TF-IDF Scores:** This is a statistical measure used to evaluate how important a word is to a document in a collection or corpus. Words that are used frequently in a single document but not frequently in the corpus receive a higher TF-IDF score.

In [13]:
# Remuve unneccessary spaces
df['description'] = df['description'].str.strip() 
# Remove start word which means description
df['description'] = df['description'].str.replace(r'leírás\n','', regex=True)


In [14]:
df['description_length'] = df['description'].apply(len)
df['word_count'] = df['description'].apply(lambda x: len(x.split()))


# Sentiment analyses
# TODO: Translatie description
# df['description_polarity'] = df['description'].apply(lambda x: TextBlob(x).sentiment.polarity)
# df['description_subjectivity'] = df['description'].apply(lambda x: TextBlob(x).sentiment.subjectivity)

In [15]:
def lemmatize_text(text):
    mytokens = text.split()
    lemmetized_tokens = [simplemma.lemmatize(x, lang='hu') for x in mytokens]
    
    return ' '.join(lemmetized_tokens)

In [16]:
# Remove non-alpahbetic characters
df['description_lemmatized'] = df['description'].str.replace('[^a-záéíóöőúüű\s]', '', regex=True)

# Lemmatize
df['description_lemmatized'] = [lemmatize_text(x) for x in df['description_lemmatized']]

# Manula lemmatization for specail cases
word_map = {'cserél':'csere', 'garantál':'garancia', 
            'állapotú':'állapot', 'állap':'állapot', 'állapotot':'állapot', 'használat':'használ', 
            'ülése':'ülés', 'elektromosan':'elektromos', 'magyarországi':'magyarország',
            'megtekinthető':'megtekintés', 'ülése':'ülés', 'km':'kilométer',
            'gépjármű':'gépkocsi', 'autó':'gépkocsi'}

for k,v in word_map.items():
    df['description_lemmatized'] = df['description_lemmatized'].str.replace(k, v)


In [17]:
# 'hungarian_stop_words' is a list of stop words in Hungarian
with open('stopwords-hu.txt', 'r', encoding='utf-8') as f:
    hungarian_stop_words = [line.strip() for line in f]

In [18]:
# Create the TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words=hungarian_stop_words, max_features=90)

# Apply the vectorizer
X = vectorizer.fit_transform(df['description_lemmatized'])

# Translate to inglis the feature names
translator = Translator()
translations = translator.translate(list(vectorizer.get_feature_names_out()), src='hu', dest='en')

# Create a dataframe from the features
tfidf_df = pd.DataFrame(X.toarray(), columns=['tfidf_'+x.text for x in translations])

/Users/davidszalai/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['stb', 'оda'] not in stop_words.
  warnings.warn(


In [19]:
tfidf_df.rename(columns={'tfidf_sports':'tfidf_sport'}, inplace=True, )

In [20]:
vectorizer.get_feature_names_out()

array(['ablak', 'alap', 'alváz', 'automata', 'belső', 'beszámít',
       'biztosítás', 'bőr', 'csere', 'csomag', 'db', 'egyeztetés', 'elad',
       'elektromos', 'elvihető', 'elöl', 'es', 'eset', 'esztétika',
       'extra', 'felszereltség', 'forgalom', 'friss', 'fényszóró',
       'fűthető', 'garancia', 'gumi', 'gyári', 'gépkocsi', 'használ',
       'hirdetés', 'hitel', 'hátsó', 'időpont', 'ig', 'jármű',
       'karbantart', 'kerül', 'kilométer', 'klíma', 'kormány', 'kulcs',
       'kér', 'köszön', 'külső', 'led', 'lehetséges', 'leinformálható',
       'légzsák', 'magyar', 'magyarország', 'megbízható', 'megkímélt',
       'megtekintés', 'minősül', 'motor', 'működik', 'műszaki',
       'navigáció', 'nyári', 'olaj', 'rendelkezik', 'rendszer',
       'rendszeres', 'stb', 'személyes', 'szerel', 'szerviz', 'szép',
       'tart', 'tartalmaz', 'telefon', 'tempomat', 'tud', 'tulajdonos',
       'téli', 'tükör', 'vezet', 'vált', 'vár', 'vásárlás', 'vásárol',
       'vég', 'állapot', 'állítható

In [21]:
tfidf_df.columns.value_counts().head(10)

tfidf_window        1
tfidf_service       1
tfidf_personal      1
tfidf_etc           1
tfidf_regular       1
tfidf_system        1
tfidf_has           1
tfidf_oil           1
tfidf_summer        1
tfidf_navigation    1
dtype: int64

In [35]:
columns_to_drop = [
            'content_info_0',
            'content_info_1',
            'content_info_2',
            'content_info_3',
            'content_info_4',
            'content_info_5',
            'content_info_6',
            'upholstery color (1)',
            'upholstery color (2)',
            'Take away from 20%',
            'Take away from 25%',
            'Can be taken away from 30%',
            'description',
            'description_lemmatized'
        ]
for x in columns_to_drop:
    print(x)

content_info_0
content_info_1
content_info_2
content_info_3
content_info_4
content_info_5
content_info_6
upholstery color (1)
upholstery color (2)
Take away from 20%
Take away from 25%
Can be taken away from 30%
description
description_lemmatized


In [22]:
# Join the new features to the original dataframe
df2 = pd.concat([df, tfidf_df], axis=1)

df2.drop(
    columns=[
     'content_info_0',
     'content_info_1',
     'content_info_2',
     'content_info_3',
     'content_info_4',
     'content_info_5',
     'upholstery color (1)',
     'upholstery color (2)',
     'content_info_6',
    'Take away from 20%',
    'Take away from 25%',
    'Can be taken away from 30%',
    'age_days',
    'description',
    'description_lemmatized'
    ],
    inplace=True
)


In [23]:
df2.head()

,price (HUF),condition,design,vehicle history,financing,clock position (km),shippable persons number,number of doors,color,own weight (kg),...,tfidf_buying,tfidf_buys,tfidf_end,tfidf_condition,tfidf_adjustable,tfidf_price,tfidf_inquire,tfidf_all,tfidf_administration,tfidf_seat
0,6895000,Kitűnő,Sedan,Elérhető autó-előélet alvázszám alapján,25,110000,5.0,4.0,fekete,1426.0,...,0.000000,0.000000,0.000000,0.136765,0.000000,0.0,0.0,0.000000,0.233220,0.000000
1,9999999,Megkímélt,Városi terepjáró (crossover),Elérhető autó-előélet alvázszám alapján,20,119300,5.0,5.0,fehér,2150.0,...,0.229360,0.000000,0.000000,0.252778,0.000000,0.0,0.0,0.000000,0.000000,0.000000
2,4995000,Kitűnő,Városi terepjáró (crossover),Elérhető autó-előélet alvázszám alapján,20,70000,5.0,5.0,kék,1200.0,...,0.000000,0.000000,0.000000,0.156297,0.000000,0.0,0.0,0.000000,0.266528,0.000000
3,3499000,Kitűnő,Városi terepjáró (crossover),Elérhető autó-előélet alvázszám alapján,100,100000,5.0,5.0,sötétkék,1176.0,...,0.128078,0.124196,0.240592,0.141155,0.142926,0.0,0.0,0.141467,0.120353,0.000000
4,12990000,Normál,Coupe,Elérhető autó-előélet alvázszám alapján,20,47336,5.0,4.0,,1490.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.117257


In [24]:
df2.set_index('link', inplace=True)

In [25]:
df_melt_corr = get_correlations(df2)
df_melt_corr

,variable1,variable2,correlation
1672,back summer tires rim diameter,summer tires rim diameter,0.997407
2029,back winter tires rim diameter,winter tires rim diameter,0.993658
3095,description_length,word_count,0.989647
1910,back winter tires apect ratio,winter tires apect ratio,0.979885
1553,back summer tires apect ratio,summer tires apect ratio,0.974668
...,...,...,...
1911,back winter tires rim diameter,winter tires apect ratio,-0.734149
2143,back summer tire width,back summer tires apect ratio,-0.745452
1554,back summer tires rim diameter,summer tires apect ratio,-0.748433
1671,back summer tires apect ratio,summer tires rim diameter,-0.765120


In [26]:
df_high_corr = df_melt_corr[df_melt_corr.correlation > 0.96]
df_high_corr

,variable1,variable2,correlation
1672,back summer tires rim diameter,summer tires rim diameter,0.997407
2029,back winter tires rim diameter,winter tires rim diameter,0.993658
3095,description_length,word_count,0.989647
1910,back winter tires apect ratio,winter tires apect ratio,0.979885
1553,back summer tires apect ratio,summer tires apect ratio,0.974668
1791,back winter tire width,winter tire width,0.962454


In [36]:
asd = pd.Series([1,2,3])
asd == asd.astype(float)

0    True
1    True
2    True
dtype: bool

In [38]:
'asd' in np.array(['as', 'a', 'asd'])

True

In [27]:
high_corr_values = df_high_corr['variable1'].values
high_corr_values

array(['back summer tires rim diameter', 'back winter tires rim diameter',
       'description_length', 'back winter tires apect ratio',
       'back summer tires apect ratio', 'back winter tire width'],
      dtype=object)

In [28]:
df_melt_corr[df_melt_corr.variable1 == 'price (HUF)']

,variable1,variable2,correlation
17,price (HUF),winter tires rim diameter,0.624799
14,price (HUF),summer tires rim diameter,0.612926
12,price (HUF),summer tire width,0.551783
15,price (HUF),winter tire width,0.544517
24,price (HUF),speaker count,0.492361
...,...,...,...
88,price (HUF),tfidf_oil,-0.120344
36,price (HUF),tfidf_exchange,-0.141112
85,price (HUF),tfidf_technical,-0.141418
13,price (HUF),summer tires apect ratio,-0.365782


In [29]:
df_melt_corr[df_melt_corr.variable2 == 'price (HUF)']

,variable1,variable2,correlation
11,initial part,price (HUF),0.932707
23,back winter tires rim diameter,price (HUF),0.691729
9,performance (kW),price (HUF),0.686560
20,back summer tires rim diameter,price (HUF),0.683748
18,back summer tire width,price (HUF),0.657869
21,back winter tire width,price (HUF),0.654324
5,own weight (kg),price (HUF),0.489578
8,cylinder capacity (cm3),price (HUF),0.433149
10,MOT is valid (days),price (HUF),0.226747
26,description_length,price (HUF),0.193590


In [30]:
# Drop 'inital part' since it almost identical to price
df2.drop(columns=['initial part'] +list(high_corr_values), inplace=True, errors='ignore')

In [31]:
# There are some extramely low prices where the price is written by mistace drop them
df2 = df2[df2['price (HUF)'] > 100000]

In [32]:
df2.to_parquet('../data/used_car_engineered_data.parquet')

In [33]:
df_hun_sattlements.to_csv('all_hun_sattlement.csv', index=False)

In [34]:
# Save descriptions to create a word cloud in EDA
text = '\n'.join(df['description'].dropna())

In [35]:
with open('descriptions.txt', 'w') as file:
    file.write(text)

In [36]:
df[tfidf_df['tfidf_sell']>0]['description'].values[:10]

array(['eladó egy új korában magyarországon, a renault miskolc-ban-ban vásárolt, és végig ott szervizelt, végig egy tulajdonos által használt, mindössze 110000 km-t futott, 90 lóerős, benzines, renault captur 0.9 tce energy alize euro 6. az autó kifogástalan műszaki állapotú, légkondicionálóval és rendeltetésszerűen működő egyéb extrafelszerelésekkel(tempomat\u200b, usb csatlakozós multimédia, elektromosan állítható-, fűthető visszapillantó tükrök, első-hátsó elektromos ablakemelők, tologatható hátsó üléssor, menetstabilizátor(esp), kipörgésgátló(asr), légzsákok(vezető, utas, oldal). a beltér tiszta, állapota tükrözi az alacsony futásteljesítményt. járműszolgáltatási platform-os lekérdezése, szervizszámlái megtekinthetőek a dokumentum fül alatt. mindkét gyári kulcskártyájával és az összes kezelési útmutatóval rendelkezik. a műszaki vizsga még több mint másfél évig érvényes. a gépkocsira egy év(igény szerint akár három) műszaki garanciát adunk. eladására új autó vásárlása miatt került s